In [ ]:
# instalação do Weights & Biases
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
# importação das bibliotecas
import logging
import tempfile
import pandas as pd
import os
import wandb
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# wandb login
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Segregação dos Dados

In [ ]:
# inicializando uma run no wandb
run = wandb.init(project="Breast Cancer TCC", job_type="split_data")

wandb: Currently logged in as: valmirfrancisco1. Use `wandb login --relogin` to force relogin


In [ ]:
# buscando o artefato de imagens no wandb
artifact = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/img_data_clean:latest', type='clean_data').file()
data_img = np.load(artifact)

# bsucando o artefato de rótulos no wandb
artifact2 = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/label_data_clean:latest', type='clean_data').file()
labels = pd.read_csv(artifact2)

In [ ]:
# utilizamos o train_test_split para dividir ps nossos 2 arefatos em arquivos de teste e treino
data_img_train, data_img_test, labels_train, labels_test = train_test_split(data_img, labels, test_size=0.2, random_state=42, stratify=labels["cancer"])

In [ ]:
if(len(data_img_train) == len(labels_train)):
  print("quantidade de valores do data_img_train é igual a quantidade de valores do labels_train")
else:
  print("quantidade de valores do data_img_train não é igual a quantidade de valores do labels_train")

quantidade de valores do data_img_train é igual a quantidade de valores do labels_train


In [ ]:
if(len(data_img_test) == len(labels_test)):
  print("quantidade de valores do data_img_test é igual a quantidade de valores do labels_test")
else:
  print("quantidade de valores do data_img_test não é igual a quantidade de valores do labels_test")

quantidade de valores do data_img_test é igual a quantidade de valores do labels_test


### Verificando os resultados da nossa divisão

In [ ]:
# essa é a contagem de valores com cancer do dataframe
filter = labels["cancer"] == 1
cancer_value = labels.loc[filter]
n = cancer_value["cancer"].count()
n

1158

In [ ]:
# essa é a contagem de valores com cancer do dataframe de treino
# para que a proporção esteja correta é necessario que ele tenha 80% de 1158, que é 926,4
filter = labels_train["cancer"] == 1
cancer_value = labels_train.loc[filter]
n = cancer_value["cancer"].count()
n

926

In [ ]:
# essa é a contagem de valores com cancer do dataframe de teste
# para que a proporção esteja correta é necessario que ele tenha 20% de 1158, que é 231,6
filter = labels_test["cancer"] == 1
cancer_value = labels_test.loc[filter]
n = cancer_value["cancer"].count()
n

232

### Salvando os dados das imagens em um arquivo numpy

In [ ]:
# treino
np.save('img_data_train.npy', data_img_train)
loaded_data_train = np.load('img_data_train.npy')

# teste
np.save('img_data_test.npy', data_img_test)
loaded_data_test = np.load('img_data_test.npy')

### Salvando os rótulos em um arquivo CSV

In [ ]:
# treino
labels_train.to_csv("label_data_train.csv", index=False)

# teste
labels_test.to_csv("label_data_test.csv", index=False)

### Enviando artefatos de treino e teste para o wandb

In [ ]:
# criando um artefato no wandb
artifact = wandb.Artifact(name="img_data_train",
                                  type="split_data",
                                  description="train data of the images")

# adiciona o arquivo no artefato
artifact.add_file('img_data_train.npy')
# salva o artefato no wandb
wandb.run.log_artifact(artifact)

# criando um artefato no wandb
artifact2 = wandb.Artifact(name="img_data_test",
                                  type="split_data",
                                  description="test data of the images")

# adiciona o arquivo no artefato
artifact2.add_file('img_data_test.npy')
# salva o artefato no wandb
wandb.run.log_artifact(artifact2)

# criando um artefato no wandb
artifact3 = wandb.Artifact(name="label_data_train",
                                  type="split_data",
                                  description="labels of the train imagens")

# adiciona o arquivo no artefato
artifact3.add_file('label_data_train.csv')
# salva o artefato no wandb
wandb.run.log_artifact(artifact3)

# criando um artefato no wandb
artifact4 = wandb.Artifact(name="label_data_test",
                                  type="split_data",
                                  description="labels of the test imagens")

# adiciona o arquivo no artefato
artifact4.add_file('label_data_test.csv')
# salva o artefato no wandb
wandb.run.log_artifact(artifact4)

<Artifact label_data_test>

In [ ]:
# termina a run
run.finish()